In [36]:
%run "/Users/ananshar/Desktop/Work/Projects/DSC214-Soccer-Passing-Network-Analysis/features_test.ipynb"

Average Distance: 0.6066217110955285
Percentage of Through Balls: 0
Pass Completion Rate: 53.98936170212766
Average Passes Per Player: 26.857142857142858
Most Advanced Player: Bojan Krkíc Pérez
Number of Passes Received by the Most Advanced Player: 9
Average X Position (excluding goalkeepers): 0.520929307116105
Average Number of Unique Passing Links per Player: 7.642857142857143
Average Team VAEP Value: 0.01752521122894877


/var/folders/gl/s_1pz2z130g9ct6hvptjf5fh0000gn/T/ipykernel_14761/1849874024.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_goalkeepers["origin_pos_x"] = df.location.apply(lambda x: _statsbomb_to_point(x)[0])
/Users/ananshar/anaconda3/envs/soccer_analysis/lib/python3.10/site-packages/socceraction/vaep.py:37: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.0465' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  prev_scores[corner_idx] = 0.046500


In [37]:
import pandas as pd
from pandas import json_normalize
from utils import read_json
import socceraction.vaep as vaep
import numpy as np
import pandas as pd
import json
import warnings
import os

In [38]:
datafolder = 'data/eventing'

matches = os.path.join(datafolder, "matches")
lineups = os.path.join(datafolder, "lineups")
events = os.path.join(datafolder, "events")

In [39]:
def get_file_paths(directory):
    season_paths = []

    for item in os.listdir(directory):
        file_path = os.path.join(directory, item)

        # Check if the item is a directory
        if os.path.isdir(file_path):
            # List all contents of the directory
            for content in os.listdir(file_path):
                content_path = os.path.join(file_path, content)

                if content.endswith(".json"):
                    season_paths.append(content_path)

    return season_paths

season_paths = get_file_paths(matches)
print(season_paths)

['data/eventing/matches/11/41.json', 'data/eventing/matches/11/40.json', 'data/eventing/matches/11/37.json', 'data/eventing/matches/11/21.json', 'data/eventing/matches/11/26.json', 'data/eventing/matches/11/27.json', 'data/eventing/matches/11/1.json', 'data/eventing/matches/11/2.json', 'data/eventing/matches/11/90.json', 'data/eventing/matches/11/24.json', 'data/eventing/matches/11/278.json', 'data/eventing/matches/11/25.json', 'data/eventing/matches/11/22.json', 'data/eventing/matches/11/38.json', 'data/eventing/matches/11/4.json', 'data/eventing/matches/11/42.json', 'data/eventing/matches/11/39.json', 'data/eventing/matches/11/23.json']


In [40]:
predictions_h5 = os.path.join("data/eventing", "predictions.h5")

spadl_h5 = os.path.join("data/eventing", "spadl-statsbomb.h5")

actiontypes = pd.read_hdf(spadl_h5, "actiontypes")
bodyparts = pd.read_hdf(spadl_h5, "bodyparts")
results = pd.read_hdf(spadl_h5, "results")
players = pd.read_hdf(spadl_h5, "players")
teams = pd.read_hdf(spadl_h5, "teams")

In [41]:
match_ids = [match["match_id"] for season_path in season_paths for match in read_json(season_path)]

In [45]:
# match_id = 68352
match_data = []

for match_id in match_ids:

    print(f"-----------Processing {match_id}.json-----------")

    actions = pd.read_hdf(spadl_h5, "actions/game_{0}".format(match_id))

    actions = (
    actions.merge(actiontypes, how="left")
    .merge(results, how="left")
    .merge(bodyparts, how="left")
    .merge(players, how="left")
    .merge(teams, how="left")
    )

    preds = pd.read_hdf(predictions_h5, "game_{0}".format(match_id))
    values = vaep.value(actions, preds.scores, preds.concedes)

    lineup = read_json(f"data/eventing/lineups/{match_id}.json")

    team_names = [lineup[0]["team_name"], lineup[1]["team_name"]]
    # team_name = teams[0]

    names_dict = {
        player["player_name"]: player["player_nickname"]
        for team in lineup
        for player in team["lineup"]
    }

    match_events = read_json(os.path.join(events, f"{match_id}.json"))
    df_events = json_normalize(match_events, sep="_").assign(match_id=match_id)

    match_row = {"match_id": match_id}

    for team in team_names:

        team_prefix = f"team{team_names.index(team) + 1}_"

        print(f"-----------Team: {team}-----------")

        df_passes = df_events[
            (df_events.type_name == "Pass")
            & (df_events.pass_outcome_name != "Injury Clearance")
            & (df_events.team_name == team)
            # & (df_events.minute < num_minutes)
        ].copy()

        goal = (1, 0.5)
        average_distance = calculate_average_distance_from_goal(df_passes, goal)
        print("Average Distance:", average_distance)

        pass_type_counts = count_pass_type(df_passes, dropna=False, normalize=True)

        recovery_passes = pass_type_counts.get("Recovery", 0)
        throw_in_passes = pass_type_counts.get("Throw-in", 0)
        free_kick_passes = pass_type_counts.get("Free Kick", 0)
        corner_passes = pass_type_counts.get("Corner", 0)
        goal_kick_passes = pass_type_counts.get("Goal Kick", 0)
        interception_passes = pass_type_counts.get("Interception", 0)
        kick_off_passes = pass_type_counts.get("Kick Off", 0)

        # Print values to verify
        print("Recovery Passes:", recovery_passes)
        print("Throw-in Passes:", throw_in_passes)
        print("Free Kick Passes:", free_kick_passes)
        print("Corner Passes:", corner_passes)
        print("Goal Kick Passes:", goal_kick_passes)
        print("Interception Passes:", interception_passes)
        print("Kick Off Passes:", kick_off_passes)

        through_ball_percentage = calculate_through_ball_percentage(
            df_passes, dropna=False
        )
        print("Percentage of Through Balls:", through_ball_percentage)

        pass_completion_rate = calculate_pass_completion_rate(df_passes)
        print("Pass Completion Rate:", pass_completion_rate)

        avg_passes_per_player = calculate_avg_passes_per_player(df_passes)
        print("Average Passes Per Player:", avg_passes_per_player)

        most_advanced_player, number_of_passes_received = (
            calculate_passes_to_most_advanced_player(df_passes)
        )
        print("Most Advanced Player:", most_advanced_player)
        print(
            "Number of Passes Received by the Most Advanced Player:",
            number_of_passes_received,
        )

        avg_x_position_non_gk = calculate_avg_x_position_excluding_goalkeepers(
            df_passes
        )
        print("Average X Position (excluding goalkeepers):", avg_x_position_non_gk)

        avg_links = average_unique_passing_links(df_passes)
        print("Average Number of Unique Passing Links per Player:", avg_links)

        def get_name_or_default(name):
            if not name or pd.isna(name):  # Checks if name is None or NaN
                return None
            # Safely gets the value from the dictionary or returns the name itself
            return (
                names_dict.get(name, name) if names_dict.get(name) is not None else name
            )

        # If available, use player's nickname instead of full name to optimize space in plot
        # df_new["pass_recipient_name"] = df_new.pass_recipient_name.apply(lambda x: None if not x else names_dict[x] if names_dict[x] else x)
        df_passes["pass_recipient_name"] = df_passes["pass_recipient_name"].apply(get_name_or_default)
        df_passes["player_name"] = df_passes.player_name.apply(
            lambda x: names_dict[x] if names_dict[x] else x
        )

        df_vaep = pd.concat([actions, preds, values], axis=1)
        # df_vaep["player_name"] = df_vaep.apply(lambda x: x["player_nickname"] if x["player_nickname"] else x["player_name"], axis=1)
        df_vaep["player_name"] = df_vaep.player_name.apply(
            lambda x: names_dict[x] if names_dict[x] else x
        )

        team_pass_value = calculate_team_pass_value(df_passes, df_vaep)
        print("Average Team VAEP Value:", team_pass_value)

        match_row.update(
            {
                f"{team_prefix}average_distance": average_distance,
                f"{team_prefix}recovery_passes": recovery_passes,
                f"{team_prefix}throw_in_passes": throw_in_passes,
                f"{team_prefix}free_kick_passes": free_kick_passes,
                f"{team_prefix}corner_passes": corner_passes,
                f"{team_prefix}goal_kick_passes": goal_kick_passes,
                f"{team_prefix}interception_passes": interception_passes,
                f"{team_prefix}kick_off_passes": kick_off_passes,
                f"{team_prefix}through_ball_percentage": through_ball_percentage,
                f"{team_prefix}pass_completion_rate": pass_completion_rate,
                f"{team_prefix}avg_passes_per_player": avg_passes_per_player,
                f"{team_prefix}most_advanced_player": most_advanced_player,
                f"{team_prefix}number_of_passes_received": number_of_passes_received,
                f"{team_prefix}avg_x_position_non_gk": avg_x_position_non_gk,
                f"{team_prefix}avg_links": avg_links,
                f"{team_prefix}team_pass_value": team_pass_value,
            }
        )

    match_data.append(match_row)

-----------Processing 69225.json-----------
-----------Team: Barcelona-----------
Average Distance: 0.5487638102935728
Recovery Passes: 0.09931506849315068
Throw-in Passes: 0.0410958904109589
Free Kick Passes: 0.032534246575342464
Corner Passes: 0.018835616438356163
Goal Kick Passes: 0.008561643835616438
Interception Passes: 0.003424657534246575
Kick Off Passes: 0.0017123287671232876
Percentage of Through Balls: 2.5684931506849313
Pass Completion Rate: 81.84931506849315


/Users/ananshar/anaconda3/envs/soccer_analysis/lib/python3.10/site-packages/socceraction/vaep.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.792453' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  prev_scores[penalty_idx] = 0.792453
/var/folders/gl/s_1pz2z130g9ct6hvptjf5fh0000gn/T/ipykernel_14761/1849874024.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_goalkeepers["origin_pos_x"] = df.location.apply(lambda x: _statsbomb_to_point(x)[0])
/var/folders/gl/s_1pz2z130g9ct6hvptjf5fh0000gn/T/ipykernel_14761/1849874024.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

Average Passes Per Player: 44.92307692307692
Most Advanced Player: Thierry Henry
Number of Passes Received by the Most Advanced Player: 47
Average X Position (excluding goalkeepers): 0.5534174041297936
Average Number of Unique Passing Links per Player: 8.307692307692308
Average Team VAEP Value: 0.015936307649282402
-----------Team: Real Madrid-----------
Average Distance: 0.6497966942958443
Recovery Passes: 0.16774193548387098
Throw-in Passes: 0.06774193548387097
Free Kick Passes: 0.05161290322580645
Corner Passes: 0.012903225806451613
Goal Kick Passes: 0.03225806451612903
Interception Passes: 0.01935483870967742
Kick Off Passes: 0.00967741935483871
Percentage of Through Balls: 1.935483870967742
Pass Completion Rate: 63.54838709677419
Average Passes Per Player: 22.142857142857142
Most Advanced Player: Miguel Palanca Fernández
Number of Passes Received by the Most Advanced Player: 19
Average X Position (excluding goalkeepers): 0.45876325088339226
Average Number of Unique Passing Links p

/var/folders/gl/s_1pz2z130g9ct6hvptjf5fh0000gn/T/ipykernel_14761/1849874024.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_goalkeepers["origin_pos_x"] = df.location.apply(lambda x: _statsbomb_to_point(x)[0])
/var/folders/gl/s_1pz2z130g9ct6hvptjf5fh0000gn/T/ipykernel_14761/1849874024.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_goalkeepers["origin_pos_x"] = df.location.apply(lambda x: _statsbomb_to_point(x)[0])
/Users/ananshar/anaconda3/envs/soccer_analysis/lib/python3.10/site-packages/

-----------Team: Barcelona-----------
Average Distance: 0.6066217110955285
Recovery Passes: 0.2074468085106383
Throw-in Passes: 0.11702127659574468
Free Kick Passes: 0.0425531914893617
Corner Passes: 0.010638297872340425
Goal Kick Passes: 0.02127659574468085
Interception Passes: 0.015957446808510637
Kick Off Passes: 0.005319148936170213
Percentage of Through Balls: 0
Pass Completion Rate: 53.98936170212766
Average Passes Per Player: 26.857142857142858
Most Advanced Player: Bojan Krkíc Pérez
Number of Passes Received by the Most Advanced Player: 9
Average X Position (excluding goalkeepers): 0.520929307116105
Average Number of Unique Passing Links per Player: 7.642857142857143
Average Team VAEP Value: 0.01752521122894877
-----------Team: Málaga-----------
Average Distance: 0.6278986426412224
Recovery Passes: 0.2214765100671141
Throw-in Passes: 0.10738255033557047
Free Kick Passes: 0.08053691275167785
Corner Passes: 0.003355704697986577
Goal Kick Passes: 0.03355704697986577
Interception P

...

/var/folders/gl/s_1pz2z130g9ct6hvptjf5fh0000gn/T/ipykernel_14761/1849874024.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_goalkeepers["origin_pos_x"] = df.location.apply(lambda x: _statsbomb_to_point(x)[0])
/var/folders/gl/s_1pz2z130g9ct6hvptjf5fh0000gn/T/ipykernel_14761/1849874024.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_goalkeepers["origin_pos_x"] = df.location.apply(lambda x: _statsbomb_to_point(x)[0])
/Users/ananshar/anaconda3/envs/soccer_analysis/lib/python3.10/site-packages/

Average Team VAEP Value: 0.00470973524797256
-----------Processing 70224.json-----------
-----------Team: Barcelona-----------
Average Distance: 0.5820862956559627
Recovery Passes: 0.06521739130434782
Throw-in Passes: 0.03188405797101449
Free Kick Passes: 0.030434782608695653
Corner Passes: 0.013043478260869565
Goal Kick Passes: 0.007246376811594203
Interception Passes: 0.0014492753623188406
Kick Off Passes: 0.0014492753623188406
Percentage of Through Balls: 0.8695652173913043
Pass Completion Rate: 85.3623188405797
Average Passes Per Player: 49.285714285714285
Most Advanced Player: Cristian Tello Herrera
Number of Passes Received by the Most Advanced Player: 5
Average X Position (excluding goalkeepers): 0.5337162849872773
Average Number of Unique Passing Links per Player: 9.285714285714286
Average Team VAEP Value: 0.017323569508536977
-----------Team: Rayo Vallecano-----------
Average Distance: 0.640718162657869
Recovery Passes: 0.11570247933884298
Throw-in Passes: 0.08264462809917356


/Users/ananshar/anaconda3/envs/soccer_analysis/lib/python3.10/site-packages/socceraction/vaep.py:37: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.0465' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  prev_scores[corner_idx] = 0.046500
/var/folders/gl/s_1pz2z130g9ct6hvptjf5fh0000gn/T/ipykernel_14761/1849874024.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_goalkeepers["origin_pos_x"] = df.location.apply(lambda x: _statsbomb_to_point(x)[0])
/var/folders/gl/s_1pz2z130g9ct6hvptjf5fh0000gn/T/ipykernel_14761/1849874024.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.

-----------Team: Barcelona-----------
Average Distance: 0.4991095945315792
Recovery Passes: 0.061696658097686374
Throw-in Passes: 0.02313624678663239
Free Kick Passes: 0.02056555269922879
Corner Passes: 0.007712082262210797
Goal Kick Passes: 0.006426735218508998
Interception Passes: 0.0012853470437017994
Kick Off Passes: 0.002570694087403599
Percentage of Through Balls: 0.7712082262210797
Pass Completion Rate: 87.53213367609256
Average Passes Per Player: 55.57142857142857
Most Advanced Player: Juan Isaac Cuenca López
Number of Passes Received by the Most Advanced Player: 37
Average X Position (excluding goalkeepers): 0.5950054680664917
Average Number of Unique Passing Links per Player: 9.428571428571429
Average Team VAEP Value: 0.012989312822902953
-----------Team: Levante UD-----------
Average Distance: 0.6561326217418993
Recovery Passes: 0.11036789297658862
Throw-in Passes: 0.0903010033444816
Free Kick Passes: 0.023411371237458192
Corner Passes: 0.0033444816053511705
Goal Kick Passes

/var/folders/gl/s_1pz2z130g9ct6hvptjf5fh0000gn/T/ipykernel_14761/1849874024.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_goalkeepers["origin_pos_x"] = df.location.apply(lambda x: _statsbomb_to_point(x)[0])
/var/folders/gl/s_1pz2z130g9ct6hvptjf5fh0000gn/T/ipykernel_14761/1849874024.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_goalkeepers["origin_pos_x"] = df.location.apply(lambda x: _statsbomb_to_point(x)[0])
/Users/ananshar/anaconda3/envs/soccer_analysis/lib/python3.10/site-packages/

In [46]:
df_matches = pd.DataFrame(match_data)
df_matches.shape

(868, 33)

In [44]:
# # match_id = 69225
# for match_id in match_ids:

#     print(f"-----------Processing {match_id}.json-----------")

#     actions = pd.read_hdf(spadl_h5, "actions/game_{0}".format(match_id))
#     actions = (
#         actions.merge(actiontypes, how="left")
#         .merge(results, how="left")
#         .merge(bodyparts, how="left")
#         .merge(players, how="left")
#         .merge(teams, how="left")
#     )

#     preds = pd.read_hdf(predictions_h5, "game_{0}".format(match_id))
#     values = vaep.value(actions, preds.scores, preds.concedes)
#     lineup = read_json(f"data/eventing/lineups/{match_id}.json")
#     team_names = [lineup[0]["team_name"], lineup[1]["team_name"]]
#     # team_name = teams[0]

#     names_dict = {
#         player["player_name"]: player["player_nickname"]
#         for team in lineup
#         for player in team["lineup"]
#     }

#     match_events = read_json(os.path.join(events, f"{match_id}.json"))
#     df_events = json_normalize(match_events, sep="_").assign(match_id=match_id)

#     for team in team_names:
#         print(f"-----------Team: {team}-----------")
#         df_passes = df_events[
#             (df_events.type_name == "Pass")
#             & (df_events.pass_outcome_name != "Injury Clearance")
#             & (df_events.team_name == team)
#             # & (df_events.minute < num_minutes)
#         ].copy()

#         # print(df_passes["pass_through_ball"].head())

#         goal = (1, 0.5)

#         average_distance = calculate_average_distance_from_goal(df_passes, goal)
#         print("Average Distance:", average_distance)

#         pass_type_counts = count_pass_type(df_passes, dropna=False, normalize=True)
#         recovery_passes = pass_type_counts.get("Recovery", 0)
#         throw_in_passes = pass_type_counts.get("Throw-in", 0)
#         free_kick_passes = pass_type_counts.get("Free Kick", 0)
#         corner_passes = pass_type_counts.get("Corner", 0)
#         goal_kick_passes = pass_type_counts.get("Goal Kick", 0)
#         interception_passes = pass_type_counts.get("Interception", 0)
#         kick_off_passes = pass_type_counts.get("Kick Off", 0)

#         # Print values to verify
#         print("Recovery Passes:", recovery_passes)
#         print("Throw-in Passes:", throw_in_passes)
#         print("Free Kick Passes:", free_kick_passes)
#         print("Corner Passes:", corner_passes)
#         print("Goal Kick Passes:", goal_kick_passes)
#         print("Interception Passes:", interception_passes)
#         print("Kick Off Passes:", kick_off_passes)

#         print(f"Match: {match_id}")

#         through_ball_percentage = calculate_through_ball_percentage(
#             df_passes, dropna=False
#         )
#         print("Percentage of Through Balls:", through_ball_percentage)

#         pass_completion_rate = calculate_pass_completion_rate(df_passes)
#         print("Pass Completion Rate:", pass_completion_rate)

#         avg_passes_per_player = calculate_avg_passes_per_player(df_passes)
#         print("Average Passes Per Player:", avg_passes_per_player)

#         most_advanced_player, number_of_passes_received = (
#             calculate_passes_to_most_advanced_player(df_passes)
#         )
#         print("Most Advanced Player:", most_advanced_player)
#         print("Number of Passes Received by the Most Advanced Player:", number_of_passes_received)

#         avg_x_position_non_gk = calculate_avg_x_position_excluding_goalkeepers(
#             df_passes
#         )
#         print("Average X Position (excluding goalkeepers):", avg_x_position_non_gk)
#         avg_links = average_unique_passing_links(df_passes)

#         print("Average Number of Unique Passing Links per Player:", avg_links)
#         def get_name_or_default(name):
#             if not name or pd.isna(name):  # Checks if name is None or NaN
#                 return None
#             # Safely gets the value from the dictionary or returns the name itself
#             return (
#                 names_dict.get(name, name) if names_dict.get(name) is not None else name
#             )

#         # If available, use player's nickname instead of full name to optimize space in plot
#         # df_new["pass_recipient_name"] = df_new.pass_recipient_name.apply(lambda x: None if not x else names_dict[x] if names_dict[x] else x)
#         df_passes["pass_recipient_name"] = df_passes["pass_recipient_name"].apply(
#             get_name_or_default
#         )

#         df_passes["player_name"] = df_passes.player_name.apply(
#             lambda x: names_dict[x] if names_dict[x] else x
#         )

#         df_vaep = pd.concat([actions, preds, values], axis=1)
#         # df_vaep["player_name"] = df_vaep.apply(lambda x: x["player_nickname"] if x["player_nickname"] else x["player_name"], axis=1)
#         df_vaep["player_name"] = df_vaep.player_name.apply(
#             lambda x: names_dict[x] if names_dict[x] else x
#         )

#         team_pass_value = calculate_team_pass_value(df_passes, df_vaep)
#         print("Average Team VAEP Value:", team_pass_value)